# Exploratory Data Analysis with SQL 

### Dataset: Netflix Dataset from Kaggle by OctopusTeam
#### Dataset Link: *https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset*

##### By: Tyler Lee
##### Date: November, 2024

___

## Importing packages, Dataset and Data Pre-processing

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('netflix_data.csv')

In [3]:
df.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,518304.0,"AT, CH, DE"
1,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1225300.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."
2,Jarhead,movie,"Biography, Drama, War",2005.0,tt0418763,7.0,212177.0,"AD, AE, AG, AL, AO, AT, AZ, BA, BG, BH, BM, BR..."
3,Unforgiven,movie,"Drama, Western",1992.0,tt0105695,8.2,445354.0,"AU, BA, BG, CZ, HR, HU, MD, ME, MK, NZ, PL, RO..."
4,Eternal Sunshine of the Spotless Mind,movie,"Drama, Romance, Sci-Fi",2004.0,tt0338013,8.3,1109205.0,"AD, AE, AG, AL, AO, AR, AU, AZ, BA, BB, BE, BG..."


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20164 entries, 0 to 20163
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               19650 non-null  object 
 1   type                20164 non-null  object 
 2   genres              19896 non-null  object 
 3   releaseYear         20149 non-null  float64
 4   imdbId              18864 non-null  object 
 5   imdbAverageRating   18713 non-null  float64
 6   imdbNumVotes        18713 non-null  float64
 7   availableCountries  20164 non-null  object 
dtypes: float64(3), object(5)
memory usage: 1.2+ MB
None


In [5]:
print(df.describe())

        releaseYear  imdbAverageRating  imdbNumVotes
count  20149.000000       18713.000000  1.871300e+04
mean    2012.946896           6.397766  3.188359e+04
std       14.415474           1.094506  1.200018e+05
min     1913.000000           1.200000  5.000000e+00
25%     2011.000000           5.700000  3.390000e+02
50%     2018.000000           6.500000  1.624000e+03
75%     2021.000000           7.200000  1.036700e+04
max     2025.000000           9.500000  2.971050e+06


In [6]:
print(df.isnull().sum())

title                  514
type                     0
genres                 268
releaseYear             15
imdbId                1300
imdbAverageRating     1451
imdbNumVotes          1451
availableCountries       0
dtype: int64


In [7]:
print(df.shape)

(20164, 8)


### Data Cleaning

Data without specific elements such as genres, titles, or IMDB id could not be replaced by most frequent value or mean value, hence it will removed from the dataset as it would not be able to serve as any insights 

In [8]:
df = df.dropna(subset=['genres'])
df = df.dropna(subset=['title'])
df = df.dropna(subset=['imdbId'])

In [9]:
# Replace missing values in 'imdbAverageRating' with the column's mean
df['imdbAverageRating'] = df['imdbAverageRating'].fillna(df['imdbAverageRating'].mean())

# Replace missing values in 'imdbNumVotes' with the column's mean
df['imdbNumVotes'] = df['imdbNumVotes'].fillna(df['imdbNumVotes'].mean())

In [10]:
# Converting Release year to Year data format
df['releaseYear'] = pd.to_datetime(df['releaseYear'], format='%Y')
df['releaseYear'] = df['releaseYear'].dt.year

In [11]:
# Rechecking for missing values and Data size
print(df.info())
print(df.isnull().sum())
print("Data size:", df.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 18863 entries, 0 to 20156
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               18863 non-null  object 
 1   type                18863 non-null  object 
 2   genres              18863 non-null  object 
 3   releaseYear         18863 non-null  int32  
 4   imdbId              18863 non-null  object 
 5   imdbAverageRating   18863 non-null  float64
 6   imdbNumVotes        18863 non-null  float64
 7   availableCountries  18863 non-null  object 
dtypes: float64(2), int32(1), object(5)
memory usage: 1.2+ MB
None
title                 0
type                  0
genres                0
releaseYear           0
imdbId                0
imdbAverageRating     0
imdbNumVotes          0
availableCountries    0
dtype: int64
Data size: (18863, 8)


### Improving Data insights with extra variable columns based on existing data

In [12]:
# Locating Total Available Countries
df['totalAvailableCountries'] = df['availableCountries'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)

In [13]:
# Categorizing IMDB Rating
def rating_category(rating):
    if rating >= 8.0:
        return 'Excellent'
    elif rating >= 6.0:
        return 'Good'
    elif rating >= 4.0:
        return 'Average'
    else:
        return 'Poor'

df['ratingCategory'] = df['imdbAverageRating'].apply(rating_category)

In [14]:
# Popularity Index to determine Data's popularity based on Number of votes and Average Rating
df['popularityIndex'] = df['imdbAverageRating'] * df['imdbNumVotes']

In [15]:
# Categorizing Popularity Index for easier understanding and interpretation
scaler = MinMaxScaler(feature_range=(0, 100))
df['popularityIndexScaled'] = scaler.fit_transform(df[['popularityIndex']])

def categorize_popularity(score):
    if score >= 80:
        return 'Extremely Popular'
    elif score >= 60:
        return 'Very Popular'
    elif score >= 40:
        return 'Popular'
    elif score >= 20:
        return 'Average Popular'
    else:
        return 'Under Popular'

df['popularityCategory'] = df['popularityIndexScaled'].apply(categorize_popularity)
df['popularityIndexScaled'] = df['popularityIndexScaled'].round(1)

In [16]:
df.head()

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,totalAvailableCountries,ratingCategory,popularityIndex,popularityIndexScaled,popularityCategory
0,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997,tt0119116,7.6,518304.0,"AT, CH, DE",3,Good,3939110.4,14.3,Under Popular
1,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003,tt0266697,8.2,1225300.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM...",68,Excellent,10047460.0,36.4,Average Popular
2,Jarhead,movie,"Biography, Drama, War",2005,tt0418763,7.0,212177.0,"AD, AE, AG, AL, AO, AT, AZ, BA, BG, BH, BM, BR...",102,Good,1485239.0,5.4,Under Popular
3,Unforgiven,movie,"Drama, Western",1992,tt0105695,8.2,445354.0,"AU, BA, BG, CZ, HR, HU, MD, ME, MK, NZ, PL, RO...",15,Excellent,3651902.8,13.2,Under Popular
4,Eternal Sunshine of the Spotless Mind,movie,"Drama, Romance, Sci-Fi",2004,tt0338013,8.3,1109205.0,"AD, AE, AG, AL, AO, AR, AU, AZ, BA, BB, BE, BG...",126,Excellent,9206401.5,33.3,Average Popular


___

## Converting Processed data to Database format

In [17]:
conn = sqlite3.connect('netflixdb.db')

df.to_sql('netflix', conn, if_exists='replace', index=False)

%load_ext sql

%sql sqlite:///netflixdb.db

In [18]:
%%sql
SELECT * FROM netflix LIMIT 5;

 * sqlite:///netflixdb.db
Done.


title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,totalAvailableCountries,ratingCategory,popularityIndex,popularityIndexScaled,popularityCategory
The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997,tt0119116,7.6,518304.0,"AT, CH, DE",3,Good,3939110.4,14.3,Under Popular
Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003,tt0266697,8.2,1225300.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM, CZ, DE, DK, EE, EG, FI, FJ, GH, GQ, GR, HR, HU, IQ, IS, IT, JO, KE, KR, KW, LB, LI, LT, LU, LV, LY, MC, MD, ME, MK, MT, MZ, NE, NG, NO, NZ, OM, PL, QA, RO, RS, SA, SC, SE, SI, SK, SM, SN, TD, TR, TZ, UA, UG, YE, ZA, ZM, ZW",68,Excellent,10047460.0,36.4,Average Popular
Jarhead,movie,"Biography, Drama, War",2005,tt0418763,7.0,212177.0,"AD, AE, AG, AL, AO, AT, AZ, BA, BG, BH, BM, BR, BS, BY, BZ, CH, CI, CM, CO, CR, CU, CV, CY, CZ, DE, DK, DO, DZ, EC, EE, EG, ES, GF, GH, GR, GT, HK, HN, HR, HU, ID, IL, IQ, IS, JM, JO, JP, KE, KR, KW, LB, LC, LI, LT, LU, LV, LY, MA, MC, MD, ME, MG, MK, ML, MT, MU, MX, MY, MZ, NG, NO, NZ, PA, PE, PF, PL, PS, PT, PY, QA, RO, RS, SA, SC, SE, SG, SI, SK, SN, SV, TC, TH, TR, TW, TZ, UA, US, UY, VE, ZA, ZM, ZW",102,Good,1485239.0,5.4,Under Popular
Unforgiven,movie,"Drama, Western",1992,tt0105695,8.2,445354.0,"AU, BA, BG, CZ, HR, HU, MD, ME, MK, NZ, PL, RO, RS, SI, SK",15,Excellent,3651902.8,13.2,Under Popular
Eternal Sunshine of the Spotless Mind,movie,"Drama, Romance, Sci-Fi",2004,tt0338013,8.3,1109205.0,"AD, AE, AG, AL, AO, AR, AU, AZ, BA, BB, BE, BG, BH, BM, BO, BR, BS, BY, BZ, CH, CI, CL, CM, CO, CR, CU, CV, CY, CZ, DK, DO, DZ, EC, EE, EG, ES, FI, FJ, FR, GB, GF, GG, GH, GI, GQ, GR, GT, HK, HN, HR, HU, ID, IE, IL, IN, IQ, IS, IT, JM, JO, JP, KE, KR, KW, LB, LC, LT, LU, LV, LY, MA, MC, MD, ME, MG, MK, ML, MT, MU, MX, MY, MZ, NE, NG, NI, NL, NO, NZ, OM, PA, PE, PF, PH, PK, PL, PS, PT, PY, QA, RO, RS, SA, SC, SE, SG, SI, SK, SM, SN, SV, TC, TH, TN, TR, TT, TW, TZ, UA, UG, US, UY, VE, YE, ZA, ZM, ZW",126,Excellent,9206401.5,33.3,Average Popular


___

## Exploratory Data Analysis

#### 1. Top 10 highest rated netflix Titles Released After the Year 2010

In [19]:
%%sql

SELECT title, imdbId, releaseYear, imdbAverageRating, ratingCategory
FROM netflix
WHERE releaseYear > 2010
ORDER BY imdbAverageRating DESC
LIMIT 10;

 * sqlite:///netflixdb.db
Done.


title,imdbId,releaseYear,imdbAverageRating,ratingCategory
Ayaanle,tt15134118,2022,9.4,Excellent
Losing Lerato 2,tt33095169,2024,9.3,Excellent
Flavours of Romania,tt13094256,2017,9.3,Excellent
Eddie's Lil' Homies,tt21361900,2024,9.3,Excellent
Mati a Zazzau,tt16762488,2020,9.2,Excellent
The Chosen,tt5622316,2017,9.2,Excellent
Our Planet,tt9253866,2019,9.2,Excellent
Shasha and Milo,tt27179112,2023,9.2,Excellent
Alone,tt8286888,2018,9.1,Excellent
Jonas,tt7799740,2016,9.1,Excellent


#### 2. Top 10 netflix shows available to the max number of countries based on Average Rating

In [20]:
%%sql

SELECT title, imdbId, totalAvailableCountries, imdbAverageRating, ratingCategory
FROM netflix
WHERE totalAvailableCountries = (SELECT MAX(totalAvailableCountries) FROM netflix)
ORDER BY imdbAverageRating DESC
LIMIT 10;

 * sqlite:///netflixdb.db
Done.


title,imdbId,totalAvailableCountries,imdbAverageRating,ratingCategory
Arcane,tt11126994,131,9.0,Excellent
Blue Eye Samurai,tt13309742,131,8.7,Excellent
The Comeback: 2004 Boston Red Sox,tt33356012,131,8.7,Excellent
Extraordinary Attorney Woo,tt20869502,131,8.6,Excellent
Derry Girls,tt7120662,131,8.5,Excellent
Queer Eye,tt7259746,131,8.5,Excellent
Formula 1: Drive to Survive,tt8289930,131,8.5,Excellent
The Queen's Gambit,tt10048342,131,8.5,Excellent
The Dragon Prince,tt8688814,131,8.3,Excellent
Car Masters: Rust to Riches,tt8893550,131,8.3,Excellent


#### 3. Top 5 Under Popular Shows with the Highest Rating

In [21]:
%%sql

SELECT title, imdbId, imdbAverageRating, ratingCategory, popularityCategory
FROM netflix
WHERE popularityCategory = "Under Popular"
ORDER BY imdbAverageRating DESC
LIMIT 5;

 * sqlite:///netflixdb.db
Done.


title,imdbId,imdbAverageRating,ratingCategory,popularityCategory
Ayaanle,tt15134118,9.4,Excellent,Under Popular
Band of Brothers,tt0185906,9.4,Excellent,Under Popular
Losing Lerato 2,tt33095169,9.3,Excellent,Under Popular
Avatar: The Last Airbender,tt0417299,9.3,Excellent,Under Popular
Flavours of Romania,tt13094256,9.3,Excellent,Under Popular


#### 4. Most imdb voted show with its available countries and Average Rating

In [22]:
%%sql

SELECT title, imdbNumVotes, imdbAverageRating, ratingCategory, totalAvailableCountries, availableCountries
FROM netflix
WHERE imdbNumVotes = (SELECT MAX(imdbNumVotes) FROM netflix);

 * sqlite:///netflixdb.db
Done.


title,imdbNumVotes,imdbAverageRating,ratingCategory,totalAvailableCountries,availableCountries
The Shawshank Redemption,2971050.0,9.3,Excellent,23,"AD, BA, BE, BG, CA, CV, CZ, ES, HR, HU, IN, JP, MD, ME, MK, MZ, NL, PL, PT, RO, RS, SI, SK"


#### 5. Netflix TV Series with the most imdb votes and its rating

In [23]:
%%sql

SELECT title, type, imdbAverageRating, imdbNumVotes 
FROM netflix 
WHERE type = "tv"
ORDER BY imdbNumVotes DESC
LIMIT 1;

 * sqlite:///netflixdb.db
Done.


title,type,imdbAverageRating,imdbNumVotes
Breaking Bad,tv,9.5,2242894.0


#### 6. The average imdb rating of all movies released on each year from 2001 to 2024

In [24]:
%%sql

SELECT releaseYear, type, ROUND(AVG(imdbAverageRating), 1) AS "Average Rating of Year", COUNT(*) AS "Total movies released"
FROM netflix
WHERE type = "movie" and releaseYear BETWEEN 2001 AND 2024
GROUP BY releaseYear, type
ORDER BY releaseYear;

 * sqlite:///netflixdb.db
Done.


releaseYear,type,Average Rating of Year,Total movies released
2001,movie,6.5,132
2002,movie,6.4,151
2003,movie,6.4,182
2004,movie,6.5,216
2005,movie,6.4,211
2006,movie,6.4,240
2007,movie,6.4,234
2008,movie,6.4,245
2009,movie,6.3,273
2010,movie,6.3,294


#### 7. 5 Lowest Rating for TV Series that contain the word "Love" in their title.

In [25]:
%%sql

SELECT title, type, imdbAverageRating, ratingCategory
FROM netflix
WHERE type = "tv" AND "title" LIKE '%Love%'
ORDER BY imdbAverageRating
LIMIT 5;

 * sqlite:///netflixdb.db
Done.


title,type,imdbAverageRating,ratingCategory
IRL: In Real Love,tv,3.3,Poor
What the Love! with Karan Johar,tv,3.7,Poor
Deep Fake Love,tv,4.4,Average
Love and Hip Hop: New York,tv,4.8,Average
Memories of Love,tv,5.0,Average


#### 8. Shows that are available in less than 10 countries and have rating higher than 8.5.

In [26]:
%%sql

SELECT title, imdbAverageRating, availableCountries, totalAvailableCountries
FROM netflix
WHERE totalAvailableCountries < 10 AND imdbAverageRating > 8.5
ORDER BY totalAvailableCountries DESC;

 * sqlite:///netflixdb.db
Done.


title,imdbAverageRating,availableCountries,totalAvailableCountries
Letterkenny,8.6,"AT, CH, DE, DK, FI, IS, LI, NO, SE",9
Fruits Basket,8.6,"HK, ID, IN, MY, PH, PK, SG, TH, TW",9
Frieren: Beyond Journey's End,8.9,"HK, ID, JP, KR, MY, PH, SG, TH, TW",9
The Lord of the Rings: The Fellowship of the Ring,8.9,"AU, CH, IN, IT, JP, KR, NZ, SM",8
The Lord of the Rings: The Two Towers,8.8,"AU, CH, IN, IT, JP, KR, NZ, SM",8
The Lord of the Rings: The Return of the King,9.0,"AU, CH, IN, IT, JP, KR, NZ, SM",8
For Our Children,8.8,"AU, CA, FJ, GB, GG, IE, NZ, US",8
The On1y One,8.8,"HK, ID, IN, PH, PK, SG, TH, TW",8
Scavengers Reign,8.6,"BM, GB, GG, GI, IE, NZ, TC, US",8
Hey Duggee,8.8,"HK, ID, MY, PH, SG, TH, TW",7


#### 9. Total movies and tv series data available in the database.

In [27]:
%%sql

SELECT type AS "Show Type", COUNT(*) AS "Total Data available"
FROM netflix
GROUP BY type;

 * sqlite:///netflixdb.db
Done.


Show Type,Total Data available
movie,14723
tv,4140


#### 10. Total Shows with Rating higher than average.

In [28]:
%%sql

SELECT COUNT(*) AS "Total Shows with Rating higher than average" 
FROM netflix
WHERE imdbAverageRating > (SELECT AVG(imdbAverageRating) FROM netflix)

 * sqlite:///netflixdb.db
Done.


Total Shows with Rating higher than average
10340


#### 11. Total count of shows with genres related to Comedy and Action while having higher rating than the overall average rating.

In [29]:
%%sql

SELECT genres, COUNT(*) AS "Count of Shows", ROUND(AVG(imdbAverageRating), 2) AS "Average Rating"
FROM netflix
WHERE genres IN (
    SELECT genres
    FROM netflix
    WHERE genres LIKE '%Comedy%' OR genres LIKE '%Action%'
    GROUP BY genres
    HAVING AVG(imdbAverageRating) > (SELECT AVG(imdbAverageRating) FROM netflix))
GROUP BY genres
ORDER BY "count of shows" DESC;

 * sqlite:///netflixdb.db
Done.


genres,Count of Shows,Average Rating
"Comedy, Drama",767,6.49
"Comedy, Drama, Romance",605,6.44
"Action, Adventure, Animation",505,6.99
"Action, Crime, Drama",325,6.44
"Comedy, Crime, Drama",160,6.67
"Action, Adventure, Drama",152,6.53
"Comedy, Documentary",130,6.53
"Comedy, Drama, Fantasy",86,6.73
"Animation, Comedy, Family",81,6.66
"Animation, Comedy, Drama",79,7.6


#### 12. Most popular genre of each year from 2001 to 2024, with the average rating of genre for the year and its available countries.

In [30]:
%%sql

WITH PopularGenre AS (
    SELECT 
        releaseYear, 
        genres, 
        AVG(imdbAverageRating) AS avg_rating, 
        SUM(totalAvailableCountries) AS totalavailcountries
    FROM netflix
    WHERE releaseYear BETWEEN 2001 AND 2024
    GROUP BY releaseYear, genres)

SELECT 
    releaseYear, 
    genres AS "Most Popular Genre", 
    ROUND(avg_rating, 2) AS "Average Rating", 
    totalavailcountries AS "Total Available Countries"
FROM PopularGenre
WHERE (releaseYear, avg_rating) IN (
    SELECT releaseYear, MAX(avg_rating)
    FROM PopularGenre
    GROUP BY releaseYear)
ORDER BY releaseYear;

 * sqlite:///netflixdb.db
Done.


releaseYear,Most Popular Genre,Average Rating,Total Available Countries
2001,"Adventure, Drama, Fantasy",8.9,8
2002,"Adventure, Drama, Fantasy",8.8,8
2003,"Adventure, Drama, Fantasy",8.5,9
2003,"Documentary, Music",8.5,1
2004,"Animation, Crime, Drama",8.7,116
2004,"Drama, Mystery",8.7,115
2005,"Documentary, Family, Reality-TV",9.1,73
2006,"Animation, Crime, Drama",8.9,44
2007,"Documentary, Drama",8.4,1
2007,"Drama, Family",8.4,87
